# Visualize 3D alignment

### Set environment

In [ ]:
import os 
import numpy as np
import open3d as o3d
from open3d.web_visualizer import draw

DATAROOT = '/data2/sgslam/scans'
SRC_SCAN = 'uc0150_00'
REF_SCAN = 'uc0150_01'
SRC_FRAME = 'frame-000050'
REF_FRAME = 'frame-000000'
PRED_FOLDER = '/data2/sfm/multi_agent'

src_depth_p = os.path.join(DATAROOT, SRC_SCAN, 'depth', SRC_FRAME+'.png')
ref_depth_p = os.path.join(DATAROOT, REF_SCAN, 'depth', REF_FRAME+'.png')

## Load pcd from depth images

In [ ]:
K = np.loadtxt(os.path.join(DATAROOT, SRC_SCAN, 'intrinsic', 'intrinsic_depth.txt')) # 
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.set_intrinsics(640, 480, K[0,0], K[1,1], K[0,2], K[1,2])

src_depth = o3d.io.read_image(src_depth_p)
ref_depth = o3d.io.read_image(ref_depth_p)
src_pcd = o3d.geometry.PointCloud.create_from_depth_image(src_depth, intrinsic)
ref_pcd = o3d.geometry.PointCloud.create_from_depth_image(ref_depth, intrinsic)
src_pcd.paint_uniform_color([1, 0.706, 0])
ref_pcd.paint_uniform_color([0, 0.651, 0.929])
print('Load {} and {}'.format(SRC_FRAME, REF_FRAME))


## Load alignment and transform

In [ ]:
from ri_looper import  read_loop_transformations
src_pose_f = os.path.join(DATAROOT, SRC_SCAN, 'pose', SRC_FRAME+'.txt')
ref_pose_f = os.path.join(DATAROOT, REF_SCAN, 'pose', REF_FRAME+'.txt')
scene_pair = '{}-{}'.format(SRC_SCAN,REF_SCAN)
pred_pose_f = os.path.join(PRED_FOLDER, scene_pair, 'pnp', SRC_FRAME+'.txt')
frame_pairs, poses = read_loop_transformations(pred_pose_f)
T_src = np.loadtxt(src_pose_f) # T_w_src
T_ref = np.loadtxt(ref_pose_f) # T_w_ref
print('src pose: ', src_pose_f)
print('ref pose: ', ref_pose_f)
pred_pose = poses[0] # T_ref_src
T_src_pred = T_ref @ pred_pose
# src_pcd.transform(T_src)
src_pcd.transform(T_src_pred)
ref_pcd.transform(T_ref)


## Visualize

In [ ]:
draw([src_pcd,ref_pcd])

if True:
    viz_dir = os.path.join(PRED_FOLDER, SRC_SCAN, 'viz', '{}-{}.ply'.format(SRC_FRAME, REF_FRAME))
    viz_pcd = o3d.geometry.PointCloud()
    viz_pcd += src_pcd
    viz_pcd += ref_pcd
    o3d.io.write_point_cloud(viz_dir, viz_pcd)
